In [1]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# sent_list = train['comment_text'].values.tolist()
# print(toxic_df[:10])


from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

train=pd.read_csv("train2.csv")


# X = train.comment_text
y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
X = train.comment_text.apply(lambda x: np.str_(x))

# train and validation set
print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
vect = TfidfVectorizer(max_features=30000)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

0         explan edit made usernam hardcor metallica fan...
1         d'aww match background colour 'm seem stuck th...
2         hey man 'm realli tri edit war 's guy constant...
3         `` ca n't make real suggest improv wonder sect...
4                             sir hero chanc rememb page 's
                                ...                        
159566    `` second time ask view complet contradict cov...
159567       asham horribl thing put talk page 128.61.19.93
159568    spitzer umm there actual articl prostitut ring...
159569    look like actual put speedi first version dele...
159570    `` ... realli n't think understand came idea b...
Name: comment_text, Length: 159571, dtype: object
        toxic  severe_toxic  obscene  threat  insult  identity_hate
0           0             0        0       0       0              0
1           0             0        0       0       0              0
2           0             0        0       0       0              0
3           0     

In [ ]:
# define functions for different vectorizors and classifiers
def test_vec_classifer(vect, clf):
        
    # create document-term matrices using the vectorizer

    
    # print the number of features that were generated
    print('Features: ', X_train_dtm.shape[1])
    
    # use Multinomial Naive Bayes to predict the star rating
    clf.fit(X_train_dtm, y_train)
    y_pred_class = clf.predict(X_test_dtm)
    
    # print the accuracy of its predictions
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
    #print('Confusion matrix: \n', metrics.confusion_matrix(y_test, y_pred_class))
    #y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
    #print('AUC: ', metrics.roc_auc_score(y_test, y_pred_prob))

In [2]:
# using binary relevance, kernel killed (consume too much memory)
from skmultilearn.problem_transform import BinaryRelevance
#from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB


# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier

#classifier = BinaryRelevance(GaussianNB())
classifier = BinaryRelevance(MultinomialNB())
 
# train
classifier.fit(X_train_dtm, y_train)
# predict
predictions = classifier.predict(X_test_dtm)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))


Accuracy =  0.905121199207881


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(X_test_dtm)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))
    print("\n")

**Processing toxic comments...**
Test accuracy is 0.9569097335372121


**Processing severe_toxic comments...**
Test accuracy is 0.9903993181761211


**Processing obscene comments...**
Test accuracy is 0.9772140475772692


**Processing threat comments...**
Test accuracy is 0.9968916852580654


**Processing insult comments...**
Test accuracy is 0.9698443335923596


**Processing identity_hate comments...**
Test accuracy is 0.9919785426014589




In [4]:
# kernel died

from sklearn.linear_model import LogisticRegression
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(MultinomialNB())
# train
classifier.fit(X_train_dtm, y_train)
# predict
predictions = classifier.predict(X_test_dtm)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.9019376833028351




In [5]:
# kernel died

# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(MultinomialNB())
# Training logistic regression model on train data
classifier.fit(X_train_dtm, y_train)
# predict
predictions = classifier.predict(X_test_dtm)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.9099591407013762




In [3]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics import accuracy_score


classifier_new = MLkNN(n_neighbors=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train_dtm).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test_dtm).toarray()
# train
classifier_new.fit(X_train_dtm, y_train)
# predict
predictions_new = classifier_new.predict(X_test_dtm)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))
print("\n")


/Users/huangraoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Accuracy =  0.8974005464617852


